In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
import torch.nn.functional as F

c:\Users\viren\miniconda3\envs\autogluon_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_data=datasets.MNIST('data',train=True,download=True, transform=transforms.ToTensor())
train, val=random_split(train_data, [55000,5000])
train_loader=DataLoader(train,batch_size=32)
val_loader=DataLoader(val,batch_size=32)

In [3]:
model=nn.Sequential(
    nn.Linear(28*28,64),
    nn.ReLU(),
    nn.Linear(64,64),
    nn.ReLU(),
    nn.Linear(64,10)
)


class LinearNetwork(nn.Module):

    def __init__(self):
        self.layer1 = nn.Linear(28*28,64)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear()

In [4]:
# class ConvolutionalNetwork(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.conv1 = nn.Conv2d(1, 6, 3, 1)
#         self.conv2 = nn.Conv2d(6, 16, 3, 1)
#         self.fc1 = nn.Linear(5*5*16, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84,10)

#     def forward(self, X):
#         X = F.relu(self.conv1(X))
#         X = F.max_pool2d(X, 2, 2)
#         X = F.relu(self.conv2(X))
#         X = F.max_pool2d(X, 2, 2)
#         X = X.view(-1, 5*5*16)
#         X = F.relu(self.fc1(X))
#         X = F.relu(self.fc2(X))
#         X = self.fc3(X)
#         return F.log_softmax(X, dim=1)

In [5]:
torch.manual_seed(42)
# model = ConvolutionalNetwork()
params=model.parameters()
optimiser=optim.SGD(params,lr=1e-2)

In [6]:
loss=nn.CrossEntropyLoss()

In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

nb_epochs=5
for epoch in range(nb_epochs):
  losses=list()
  accuracies=list()
  for batch in train_loader:
    x,y=batch
    b=x.size(0)
    x=x.view(b,-1)
    l=model(x)
    j=loss(l,y)
    model.zero_grad()
    j.backward()
    optimiser.step()
    losses.append(j.item())
    accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())
  print(f'Epoch {epoch+1}, train loss: {torch.tensor(losses).mean():.2f}, training accuracy: {torch.tensor(accuracies).mean():.2f}')
  losses=list()
  for batch in train_loader:
    x,y=batch
    b=x.size(0)
    x=x.view(b,-1)
    with torch.no_grad():
      l=model(x)
    j=loss(l,y)
    losses.append(j.item())
    accuracies.append(y.eq(l.detach().argmax(dim=1).cpu()).float().mean())
  print(f'Epoch {epoch+1}, train loss: {torch.tensor(losses).mean():.2f}, training accuracy: {torch.tensor(accuracies).mean():.2f}')

Epoch 1, train loss: 0.11, training accuracy: 0.97
Epoch 1, train loss: 0.11, training accuracy: 0.97
Epoch 2, train loss: 0.11, training accuracy: 0.97
Epoch 2, train loss: 0.10, training accuracy: 0.97
Epoch 3, train loss: 0.10, training accuracy: 0.97
Epoch 3, train loss: 0.10, training accuracy: 0.97
Epoch 4, train loss: 0.10, training accuracy: 0.97
Epoch 4, train loss: 0.09, training accuracy: 0.97
Epoch 5, train loss: 0.09, training accuracy: 0.97
Epoch 5, train loss: 0.09, training accuracy: 0.97


In [10]:
torch.save(model, '../saved_models/mnist_pytorch.pt')

In [11]:
model2 = torch.load('../saved_models/mnist_pytorch.pt')

In [21]:
import numpy 
import cv2

img_8 = cv2.imread('../mnist_images/8/8_0008.png')
print(img_8.shape)
img_8 = cv2.cvtColor(img_8,cv2.COLOR_RGB2GRAY)
print(img_8.shape)
img_8 = img_8.reshape(28*28)
print(img_8.shape)
img_8_tensor = torch.tensor(img_8)
model2.forward(img_8_tensor)

(28, 28, 3)
(28, 28)
(784,)


RuntimeError: expected scalar type Byte but found Float